In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
%reload_ext autoreload

In [3]:
import logging
import csv
import sys
from pathlib import Path
import subprocess


In [4]:
logger = logging.getLogger(__name__)
logger.root.setLevel('DEBUG')

In [5]:
def csv_to_list(file):
    '''read csv file `file` into a list
    
    Guess the CSV dialect (e.g. tsv, csv, etc.)
    
    Returns `list`'''
    csvFile = Path(file).expanduser().absolute()
    file_csv = []
    
    with open(csvFile, 'r') as file:
        dialect = csv.Sniffer().sniff(file.read(1024))
        file.seek(0)
        reader = csv.reader(file, dialect)
        for row in reader:
            file_csv.append(row)

    return file_csv

In [6]:
def map_headers(csv_list, expected_headers=[]):
    '''map row 0 of a csv as formatted as a list to a dictionary of expected header values'''
    missing_headers = []
    header_map = {}
    
    csvHeader = csv_list[0]
    
    logger.debug('checking for missing headers')
    for each in expected_headers:
        if each not in csvHeader:
            missing_headers.append(each)
            
    if len(missing_headers) > 0:
        logging.warning(f'missing expected headers: {missing_headers}')
    for index, value in enumerate(csvHeader):
        if value in expected_headers:
            header_map[value] = index
        
    logging.debug('completed mapping')
    return(header_map, missing_headers)

In [8]:
def do_exit(e='unknown error in unknown module: BSoD!', exit_status=0, testing=False):
    logging.info(f'exited before completion with exit code {exit_status}')
    print('program exited due to errors')
    print(e)
    if not testing:
        sys.exit(exit_status)

In [173]:
class student_path(gd_path):
    def __init__(self, class_of=None, id_number=None, name=None):
        super(student_path, self).__init__()
        self.path_parts = {'ClassOf': None, 'id_number': None, 'name': None}
        self.class_of = class_of
        self.name = name
        self.id_number = id_number
        
    @property
    def class_of(self):
        return self._class_of
    
    @class_of.setter
    def class_of(self, class_of):
        if not class_of:
            self._class_of = None
        else:    
            if not isinstance(class_of, int):
                raise TypeError('class_of must be of type `int`')
        self.path_parts['ClassOf'] = f'ClassOf-{class_of}'
        self._class_of = class_of
        
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, name):
        if not name:
            self._name = None
        else:
            if not isinstance (name, str):
                raise TypeError('name must be of type `str`')
        self.path_parts['name'] = name
        self._name = name
        
    @property
    def id_number(self):
        return self._id_number
    
    @id_number.setter
    def id_number(self, number):
        if not number:
            self._id_number = None
        else:
            if not isinstance (number, int):
                raise TypeError('id_number must be of type `int`')
        self.path_parts['id_number'] = number
        self._id_number = number
        
    @property 
    def student_dir_name(self):
        d = f"/{self.path_parts['ClassOf']}/{self.path_parts['name']} - {self.path_parts['id_number']}"
        return Path(d)
    

                                

In [177]:
s = student_path(id_number=12345)
s.class_of = 2030
s.name = "Eggs, Spam"

In [178]:
print(s.path_parts)

{'ClassOf': 'ClassOf-2030', 'id_number': 12345, 'name': 'Eggs, Spam'}


In [179]:
s.student_dir_name

PosixPath('/ClassOf-2030/Eggs, Spam - 12345')

In [87]:
class gd_path():
    def __init__(self, path=None):
        '''google drive path class
        
        Attributes:
            path(`str`): path to google drive shared drive'''
        self.path = path
        self._file_base = 'https://drive.google.com/file/d/'
        self._dir_base = 'https://drive.google.com/drive/folders/'
#         self.webview_link
    
    @property
    def path(self):
        return self._path
    
    @path.setter
    def path(self, path):
        if not path:
            self._path = None
        else:
            self._path = Path(path).expanduser().absolute()

    @property
    def webview_link(self):
        self._webview_link = None
        try:
            item_id = self.get_xattr('user.drive.id')
        except FileNotFoundError as e:
            logging.debug(f'{e}')
            return None
        except ChildProcessError as e:
            logging.debug(f'{e}')
            return None

        if len(item_id) < 1:
            return None
        else:
            item_id = item_id[0]
        
        
        if self.path.is_dir():
            self._webview_link = f'{self._dir_base}{item_id}'
        if self.path.is_file():
            self._webview_link = f'{self._file_base}{item_id}'
        return self._webview_link
            
    def check_parent(self, expected):
        '''checks if the parent matches the expected parent'''
        if self.path.parents[0].name == expected:
            return True
        else:
            return False
        
    def get_xattr(self, attribute, file=None):
        '''get the extended attributes of a file or directory
        Args:
            file(`str` or Path): path to file
            attribute('`str`'): attribute key to access

        Returns:
            `list` - attribute or key: attribute paris

        Raises:
            FileNotFoundError - file or directory does not exist
            ChildProcessError - xattr utility exits with non-zero code 
                This is common for files that have no extended attributes or do not
                have the requested attribute'''
        if not file:
            file = self.path
        else:
            file = Path(file).absolute()
            
        attributes = []
        if not file.exists():
            raise FileNotFoundError

        p = subprocess.Popen(f'xattr -p  {attribute} "{file.resolve()}"', shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        for line in p.stdout.readlines():
            attributes.append(line.decode("utf-8").strip())
    #         attributes = attributes + line.decode("utf-8").strip()
        retval = p.wait()
        if retval != 0:
            raise ChildProcessError(f'xattr exited with value: {retval}')
        return attributes        

In [88]:
g = gd_path()

In [ ]:
def main():
    
    csv_file = Path('./student.export.csv.text')
    
    drive_path = Path('/Volumes/GoogleDrive/Shared drives/IT Blabla I/')
    
    expected_headers = ['LastFirst', 'ClassOf', 'Student_Number']
    expected_headers ={'LastFirst': str, 'ClassOf': int, 'Student_Number': int}
    
    ############
    
    logging.debug('starting up')
    try:
        csv_list = csv_to_list(csv_file)
    except (FileNotFoundError, OSError, IOError) as e:
        logging.error(f'could not read csv file: {csv_file}')
        logging.error(f'{e}')
        do_exit(e, 1)
    
    header_map, missing_headers = map_headers(csv_list, expected_headers.keys())
    
    if len(missing_headers) > 0:
        do_exit(f'{csv_file.name} is missing one or more headers:\n\t{missing_headers}', 1)
    
    
    
    return
    


In [ ]:
f = main()

In [ ]:
print( f)